In [1]:
import numpy as np
import json

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [2]:
#!pip install tensorflow

In [3]:
vgg16 = VGG16(weights = 'imagenet')

In [4]:
vgg16.save('vgg16.h5')

In [5]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    return x             # preprocessed image

In [6]:
y = prepare_image("C:/Users/admin/Desktop/Computer Vision + Django/firstCarCheck.jpg")

In [7]:
preds = vgg16.predict(y)

In [10]:
preds.shape

(1, 1000)

In [11]:
CLASS_INDEX = None

In [12]:
CLASS_INDEX_PATH = 'C:/Users/admin/Desktop/Computer Vision + Django/imagenet_class_index.json'

In [13]:
def get_predictions(preds, top = 5):
    global CLASS_INDEX
    
    # Load the json file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    # get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1] # argsort() - It returns an array of indices of the same shape as arr that would sort the array.
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i], ) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse = True)
        results.append(result)
    return results

In [14]:
print(get_predictions(preds, top=5))

[[('n03770679', 'minivan', 0.40141067), ('n03977966', 'police_van', 0.33341742), ('n03769881', 'minibus', 0.11062949), ('n03417042', 'garbage_truck', 0.020466868), ('n03796401', 'moving_van', 0.010222757)]]


In [15]:
# First Check - category list creation

In [29]:
from collections import Counter, defaultdict
#The ‘collections’ module in Python that implements special container datatypes. 
#These provide alternatives to Python’s general-purpose built-in containers.
import os

In [30]:
#!pip install collection

In [31]:
print(collections)

<module 'collections' from 'C:\\Users\\admin\\anaconda3\\lib\\collections\\__init__.py'>


In [32]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('D:/Car_damage_image_dataset/data1a/training/01-whole')
    for i, img_path in enumerate(img_list):
        img = prepare_image('D:/Car_damage_image_dataset/data1a/training/01-whole/' + img_path)
        out = vgg16.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print(i, '/', len(img_list), 'complete')
    return Counter(d)

In [33]:
cat_counter = get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [34]:
cat_counter

Counter({('n04252225', 'snowplow'): 7.086041108472273,
         ('n03478589', 'half_track'): 0.8668188957963139,
         ('n04461696', 'tow_truck'): 18.77435466228053,
         ('n03126707', 'crane'): 0.10376830771565437,
         ('n03417042', 'garbage_truck'): 1.4159718877635896,
         ('n02701002', 'ambulance'): 5.562828415306285,
         ('n03977966', 'police_van'): 10.42101331037702,
         ('n03345487', 'fire_engine'): 2.669070662581362,
         ('n03670208', 'limousine'): 53.60918285558,
         ('n03594945', 'jeep'): 30.02314594644122,
         ('n03930630', 'pickup'): 65.00296038511442,
         ('n04467665', 'trailer_truck'): 3.2392254944425076,
         ('n02814533', 'beach_wagon'): 67.81619649667118,
         ('n03770679', 'minivan'): 132.0488884919905,
         ('n02974003', 'car_wheel'): 17.17670764392824,
         ('n03100240', 'convertible'): 91.41386856616009,
         ('n03459775', 'grille'): 18.05061048878997,
         ('n04285008', 'sports_car'): 116.250826

In [35]:
import pickle as pk

In [38]:
# with open('cat_counter.pk', 'wb') as f:
#     pk.dump(cat_counter, f, -1)

In [39]:
# This piece of code will be used to load already saved cat_counter
#with open('cat_counter.pk', 'rb') as f:
#   cat_counter = pk.load(f)

In [40]:
cat_list = [k for k, v in cat_counter.most_common()[:27]]

In [41]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

## First Check - Car Or Not

In [42]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg16.predict(img)
    top = get_predictions(out, top=5)
    print("Validating whether the picture is of a car or not.....")
    for j in top[0]:
        if j[0:2] in cat_list:
            print(j[0:2])
            return "Validation complete - proceed for further evaluation"
    return "Are you sure that this is a picture of a car? Please upload another picture(try a different angle or lightning)"

In [49]:
car_categories_gate("C:/Users/admin/Desktop/Computer Vision + Django/firstCarCheck3.jpg", cat_list)

Validating whether the picture is of a car or not.....
('n03417042', 'garbage_truck')


'Validation complete - proceed for further evaluation'